In [306]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import pickle
pd.options.mode.chained_assignment = None
pd.set_option('max_columns', None)
%matplotlib inline

##Notes
1. Total 2888 Properties and 21 attributes
2. Total Interactions : 170611 
3. Interactions data is from '2017-02-01 10:03:16' to '2017-04-13 23:57:02'
4. Total properties which has photos : 17866

In [307]:
prop_df = pd.read_csv('/home/kiran/D_Drive_backup/DataWorld/Data_Science/Hackathon/resources/Dataset.csv')

In [308]:
prop_df.head()

,property_id,type,activation_date,bathroom,floor,total_floor,furnishing,gym,latitude,longitude,lease_type,lift,locality,parking,property_age,property_size,swimming_pool,pin_code,rent,deposit,building_type
0,ff808081469fd6e20146a5af948000ea,BHK2,09-03-2017 14:36,1,3,4.0,SEMI_FURNISHED,1,12.876174,77.596571,FAMILY,1,Hulimavu,BOTH,2,850,1,560076.0,12000,120000,AP
1,ff8080814702d3d10147068359d200cd,BHK2,07-03-2017 12:02,2,4,11.0,SEMI_FURNISHED,1,13.018444,77.678122,FAMILY,1,Ramamurthy Nagar,BOTH,1,1233,1,560016.0,20000,150000,AP
2,ff808081470c645401470fb03f5800a6,BHK2,10-03-2017 13:43,2,0,4.0,NOT_FURNISHED,1,12.975072,77.665865,ANYONE,1,GM Palya,FOUR_WHEELER,0,1200,0,560075.0,15000,75000,AP
3,ff808081470c6454014715eaa5960281,BHK3,09-03-2017 22:16,2,3,4.0,SEMI_FURNISHED,0,12.888169,77.591282,ANYONE,0,Arakere,BOTH,1,1300,0,560076.0,17000,150000,AP
4,ff808081474aa867014771a0298f0aa6,BHK1,15-03-2017 18:29,1,1,2.0,SEMI_FURNISHED,0,12.990243,77.712962,ANYONE,0,Hoodi,BOTH,4,450,0,560048.0,6500,40000,IF


In [309]:
prop_df.shape

(28888, 21)

In [310]:
prop_df.describe()

,bathroom,floor,total_floor,gym,latitude,longitude,lift,property_age,property_size,swimming_pool,pin_code,rent,deposit
count,28888.000000,28888.000000,28845.000000,28888.000000,28888.000000,28888.000000,28888.000000,28888.000000,28888.000000,28888.000000,2.856300e+04,28888.000000,2.888800e+04
mean,1.795867,1.786659,3.518807,0.165951,12.957859,77.605186,0.279009,4.380539,966.338999,0.142585,5.615600e+05,14430.768797,1.233257e+05
std,0.778676,2.361768,3.362536,0.372043,0.219500,0.921003,0.448520,7.458914,619.390016,0.349655,8.979452e+04,9055.133718,1.255028e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1000.000000,0.000000e+00
25%,1.000000,0.000000,2.000000,0.000000,12.906324,77.563295,0.000000,0.000000,600.000000,0.000000,5.600400e+05,8000.000000,5.000000e+04
50%,2.000000,1.000000,3.000000,0.000000,12.953939,77.614568,0.000000,3.000000,900.000000,0.000000,5.600670e+05,12500.000000,1.000000e+05
75%,2.000000,2.000000,4.000000,0.000000,13.009369,77.669235,1.000000,6.000000,1200.000000,0.000000,5.600840e+05,18000.000000,1.500000e+05
max,22.000000,123.000000,38.000000,1.000000,21.176277,80.256293,1.000000,800.000000,40000.000000,1.000000,5.601000e+06,75000.000000,4.200000e+06


In [311]:
prop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28888 entries, 0 to 28887
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_id      28888 non-null  object 
 1   type             28888 non-null  object 
 2   activation_date  28888 non-null  object 
 3   bathroom         28888 non-null  int64  
 4   floor            28888 non-null  int64  
 5   total_floor      28845 non-null  float64
 6   furnishing       28888 non-null  object 
 7   gym              28888 non-null  int64  
 8   latitude         28888 non-null  float64
 9   longitude        28888 non-null  float64
 10  lease_type       28888 non-null  object 
 11  lift             28888 non-null  int64  
 12  locality         28888 non-null  object 
 13  parking          28888 non-null  object 
 14  property_age     28888 non-null  int64  
 15  property_size    28888 non-null  int64  
 16  swimming_pool    28888 non-null  int64  
 17  pin_code    

In [312]:
prop_df[prop_df['floor'] > prop_df['total_floor']].head()

,property_id,type,activation_date,bathroom,floor,total_floor,furnishing,gym,latitude,longitude,lease_type,lift,locality,parking,property_age,property_size,swimming_pool,pin_code,rent,deposit,building_type
300,ff8081814ca86486014cb20612c61e88,BHK1,11-03-2017 15:30,1,2,1.0,SEMI_FURNISHED,0,13.032645,77.553893,ANYONE,0,Yeshwanthpur,TWO_WHEELER,0,500,0,NaN,4500,45000,AP
341,ff8081814cb771fb014cd04836924e4c,BHK1,20-03-2017 12:24,1,3,2.0,SEMI_FURNISHED,0,12.890547,77.625480,ANYONE,0,Hongasandra,TWO_WHEELER,1,650,0,560076.0,6700,40000,IF
1909,ff808181510fead40151141ecc8d1751,BHK3,21-03-2017 16:24,3,4,2.0,SEMI_FURNISHED,0,13.003718,77.606345,FAMILY,0,Benson Town,TWO_WHEELER,4,1200,0,560046.0,25000,300000,IF
2187,ff8081815185752901518be9055c4957,BHK3,20-02-2017 17:57,2,3,1.0,SEMI_FURNISHED,0,12.941445,77.556525,ANYONE,0,Banashankari,BOTH,3,1650,0,560050.0,23500,150000,IF
7802,ff80818159207995015920ec36230a01,BHK4,05-03-2017 15:18,4,123,3.0,SEMI_FURNISHED,1,12.950258,77.695801,FAMILY,0,Marathahalli,BOTH,5,3822,1,560037.0,50000,250000,IH


In [313]:
prop_df.isnull().sum()

property_id          0
type                 0
activation_date      0
bathroom             0
floor                0
total_floor         43
furnishing           0
gym                  0
latitude             0
longitude            0
lease_type           0
lift                 0
locality             0
parking              0
property_age         0
property_size        0
swimming_pool        0
pin_code           325
rent                 0
deposit              0
building_type       79
dtype: int64

In [314]:
prop_df['total_floor'].fillna(prop_df['floor'],inplace=True)

In [315]:
prop_df.isnull().sum()

property_id          0
type                 0
activation_date      0
bathroom             0
floor                0
total_floor          0
furnishing           0
gym                  0
latitude             0
longitude            0
lease_type           0
lift                 0
locality             0
parking              0
property_age         0
property_size        0
swimming_pool        0
pin_code           325
rent                 0
deposit              0
building_type       79
dtype: int64

In [316]:
int_df = pd.read_csv('/home/kiran/D_Drive_backup/DataWorld/Data_Science/Hackathon/resources/Interactions.csv')

In [317]:
int_df.head()

,property_id,request_date
0,ff808081469fd6e20146a5af948000ea,2017-03-10 17:42:34
1,ff808081469fd6e20146a5af948000ea,2017-03-09 15:51:17
2,ff808081469fd6e20146a5af948000ea,2017-03-10 17:30:22
3,ff808081469fd6e20146a5af948000ea,2017-03-11 17:48:46
4,ff8080814702d3d10147068359d200cd,2017-03-30 19:59:15


In [318]:
int_df.describe()

,property_id,request_date
count,170611,170611
unique,22779,166503
top,ff80818159e061800159e4614bf4368d,2017-04-03 13:17:51
freq,193,3


In [319]:
int_df['request_date'].min()

'2017-02-01 10:03:16'

In [320]:
int_df['request_date'].max()

'2017-04-13 23:57:02'

In [321]:
photos_raw_df  = pd.read_csv('/home/kiran/D_Drive_backup/DataWorld/Data_Science/Hackathon/resources/Photos.tsv', sep ='\t')

In [322]:
photos_df = photos_raw_df[photos_raw_df['photo_urls'].notnull()]

In [323]:
photos_df.head()

,property_id,photo_urls
0,ff808081469fd6e20146a5af948000ea,"[{\title\"":\""Balcony\"",\""name\"":\""IMG_20131006..."
2,ff808081470c645401470fb03f5800a6,"[{\title\"":\""Bedroom\"",\""name\"":\""ff808081470c..."
3,ff808081470c6454014715eaa5960281,"[{\title\"":\""Bedroom\"",\""name\"":\""Screenshot_7..."
4,ff808081474aa867014771a0298f0aa6,"[{\title\"":\""Other\"",\""name\"":\""ff808081474aa8..."
5,ff8080814780f30801478d14132f03f4,"[{\title\"":\""\"",\""name\"":\""files[]\"",\""imagesM..."


In [324]:
photos_df['photo_urls'] = photos_df['photo_urls'].str.replace('\\','',regex=False)

In [325]:
photos_df['photo_urls'] = photos_df['photo_urls'].str.replace('{title','{"title',regex=False)

In [326]:
photos_df['photo_count'] = photos_df['photo_urls'].apply(lambda x : x.count('title'))

In [327]:
photos_df[photos_df['property_id'] ==  'ff8081815b1fdda8015b2000369205fd']

,property_id,photo_urls,photo_count
28885,ff8081815b1fdda8015b2000369205fd,"[{""title"":"""",""name"":""files[]"",""imagesMap"":{""or...",6


### Validation on the source file such that count is correct : 
grep 'ff8081815b1fdda8015b2000369205fd' Photos.tsv | grep -o 'title' | wc -l

6


In [328]:
photos_df.shape

(17866, 3)

In [329]:
photos_df['property_id'].describe()

count                                17866
unique                               17866
top       ff808081469fd6e20146a5af948000ea
freq                                     1
Name: property_id, dtype: object

In [330]:
prop_photo_df = pd.merge(prop_df, photos_df, how='left', on = 'property_id')

In [331]:
master_df = pd.merge(prop_photo_df, int_df, how='left', on = 'property_id')

In [332]:
master_df.drop(['photo_urls','longitude','latitude','pin_code','locality','building_type','bathroom'], axis=1,inplace=True)

In [333]:
master_df['photo_count'].fillna(0, inplace=True)

In [334]:
master_df.photo_count = master_df.photo_count.astype(int)

In [335]:
master_df['type']= master_df['type'].replace('BHK2',2).replace('BHK1',1).replace('BHK3',3).replace('RK1',0.5).replace('BHK4',4).replace('BHK4PLUS',5)

In [336]:
master_df['furnishing'].value_counts()

SEMI_FURNISHED     139768
NOT_FURNISHED       28679
FULLY_FURNISHED      8273
Name: furnishing, dtype: int64

In [337]:
master_df['furnishing'] = master_df['furnishing'].replace('NOT_FURNISHED',0).replace('SEMI_FURNISHED',1).replace('FULLY_FURNISHED',2)

In [338]:
master_df['lease_type'].value_counts()

ANYONE      87285
FAMILY      76558
BACHELOR    12667
COMPANY       210
Name: lease_type, dtype: int64

In [339]:
master_df['parking'].value_counts()

TWO_WHEELER     85495
BOTH            73794
NONE            10492
FOUR_WHEELER     6939
Name: parking, dtype: int64

In [340]:
col = ['lease_type','parking']

In [341]:
for c in col:
    lt_dummies = pd.get_dummies(master_df[c],prefix=c)
    master_df.drop(columns=[c],axis=1, inplace=True)
    master_df = master_df.join(lt_dummies)

In [342]:
master_df.columns

Index(['property_id', 'type', 'activation_date', 'floor', 'total_floor',
       'furnishing', 'gym', 'lift', 'property_age', 'property_size',
       'swimming_pool', 'rent', 'deposit', 'photo_count', 'request_date',
       'lease_type_ANYONE', 'lease_type_BACHELOR', 'lease_type_COMPANY',
       'lease_type_FAMILY', 'parking_BOTH', 'parking_FOUR_WHEELER',
       'parking_NONE', 'parking_TWO_WHEELER'],
      dtype='object')

In [343]:
master_df.shape

(176720, 23)

In [344]:
pa = master_df["property_age"].quantile(0.99)

In [345]:
master_df["property_age"] = np.where(master_df["property_age"] > pa, master_df["property_age"], pa)

In [346]:
master_df = master_df.groupby(['property_id', 'type', 'activation_date','floor',
       'total_floor', 'furnishing', 'gym', 'lift', 'property_age',
       'property_size', 'swimming_pool', 'rent', 'deposit', 'photo_count',
       'lease_type_ANYONE', 'lease_type_BACHELOR',
       'lease_type_COMPANY', 'lease_type_FAMILY', 'parking_BOTH',
       'parking_FOUR_WHEELER', 'parking_NONE', 'parking_TWO_WHEELER']).size().reset_index(name='clicks')

In [347]:
master_df.shape

(28888, 23)

In [348]:
master_df["floor"].quantile(0.99)

12.0

In [349]:
master_df["floor"] = np.where(master_df["floor"] > master_df["total_floor"], master_df["total_floor"], master_df["floor"])

In [350]:
d = master_df["deposit"].quantile(0.99)

In [351]:
master_df["deposit"] = np.where(master_df["deposit"] > d, master_df["deposit"], d)

In [352]:
master_df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,type,floor,total_floor,furnishing,gym,lift,property_age,property_size,swimming_pool,rent,deposit,photo_count,lease_type_ANYONE,lease_type_BACHELOR,lease_type_COMPANY,lease_type_FAMILY,parking_BOTH,parking_FOUR_WHEELER,parking_NONE,parking_TWO_WHEELER,clicks
count,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000,28888.00000
mean,1.90423,1.78115,3.51686,0.88573,0.16595,0.27901,24.10271,966.33900,0.14259,14430.76880,525068.49557,3.46140,0.44036,0.04192,0.00246,0.51527,0.47802,0.03448,0.04421,0.44330,6.11742
std,0.81030,2.24928,3.36294,0.45756,0.37204,0.44852,5.36012,619.39002,0.34966,9055.13372,67073.70713,4.23072,0.49644,0.20041,0.04952,0.49978,0.49953,0.18246,0.20555,0.49678,9.58403
min,0.50000,0.00000,0.00000,0.00000,0.00000,0.00000,24.00000,0.00000,0.00000,1000.00000,521300.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
25%,1.00000,0.00000,2.00000,1.00000,0.00000,0.00000,24.00000,600.00000,0.00000,8000.00000,521300.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000
50%,2.00000,1.00000,3.00000,1.00000,0.00000,0.00000,24.00000,900.00000,0.00000,12500.00000,521300.00000,2.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,3.00000
75%,2.00000,2.00000,4.00000,1.00000,0.00000,1.00000,24.00000,1200.00000,0.00000,18000.00000,521300.00000,6.00000,1.00000,0.00000,0.00000,1.00000,1.00000,0.00000,0.00000,1.00000,7.00000
max,5.00000,28.00000,38.00000,2.00000,1.00000,1.00000,800.00000,40000.00000,1.00000,75000.00000,4200000.00000,45.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,193.00000


In [353]:
master_df[master_df["property_id"] == 'ff808081469fd6e20146a5af948000ea']

,property_id,type,activation_date,floor,total_floor,furnishing,gym,lift,property_age,property_size,swimming_pool,rent,deposit,photo_count,lease_type_ANYONE,lease_type_BACHELOR,lease_type_COMPANY,lease_type_FAMILY,parking_BOTH,parking_FOUR_WHEELER,parking_NONE,parking_TWO_WHEELER,clicks
0,ff808081469fd6e20146a5af948000ea,2.0,09-03-2017 14:36,3.0,4.0,1,1,1,24.0,850,1,12000,521300.0,7,0,0,0,1,1,0,0,0,4


In [354]:
master_df.columns

Index(['property_id', 'type', 'activation_date', 'floor', 'total_floor',
       'furnishing', 'gym', 'lift', 'property_age', 'property_size',
       'swimming_pool', 'rent', 'deposit', 'photo_count', 'lease_type_ANYONE',
       'lease_type_BACHELOR', 'lease_type_COMPANY', 'lease_type_FAMILY',
       'parking_BOTH', 'parking_FOUR_WHEELER', 'parking_NONE',
       'parking_TWO_WHEELER', 'clicks'],
      dtype='object')

In [355]:
feature_cols = ['type', 'floor',
       'total_floor', 'furnishing', 'gym', 'lift', 'property_age',
       'property_size', 'swimming_pool', 'rent', 'deposit', 'photo_count',
       'lease_type_ANYONE', 'lease_type_BACHELOR', 'lease_type_COMPANY',
       'lease_type_FAMILY', 'parking_BOTH', 'parking_FOUR_WHEELER',
       'parking_NONE', 'parking_TWO_WHEELER']

In [356]:
x = master_df[feature_cols]
y = master_df['clicks']

In [357]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [358]:
linreg = LinearRegression()
linreg.fit(x_train,y_train)

LinearRegression()

In [359]:
linreg.intercept_

9.248971724383935

In [360]:
linreg.predict([[2.0,
 3.0,
 4.0,
 1.0,
 1.0,
 1.0,
 2.0,
 850.0,
 1.0,
 12000.0,
 120000.0,
 7.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0]])

array([5.06243981])

In [361]:
linreg.coef_

array([-2.61611740e+00, -1.25233152e-01,  5.38136351e-02,  7.10407041e-01,
       -5.96668655e-02, -2.85993029e-01,  1.59261334e-03, -1.67791577e-04,
       -1.01584091e+00,  5.43741462e-05, -6.85108171e-08,  3.34916632e-01,
        8.40307912e-01,  2.33695998e+00, -2.69828887e+00, -4.78979019e-01,
       -5.13026692e-01,  6.54779604e-01,  4.42499390e-01, -5.84252302e-01])

In [362]:
np.sqrt(mean_squared_error(y_train,linreg.predict(x_train)))

9.3604629934645

In [363]:
np.sqrt(mean_squared_error(y_test,linreg.predict(x_test)))

8.808929747276835

In [364]:
linreg.score(x_train,y_train)

0.06650215311783092

In [365]:
linreg.score(x_test,y_test)

0.07364891133115603

In [366]:
pickle.dump(linreg, open('click-prediction-model.pkl','wb'))

In [367]:
model = pickle.load(open('click-prediction-model.pkl','rb'))

In [368]:
master_df.corr()

,type,floor,total_floor,furnishing,gym,lift,property_age,property_size,swimming_pool,rent,deposit,photo_count,lease_type_ANYONE,lease_type_BACHELOR,lease_type_COMPANY,lease_type_FAMILY,parking_BOTH,parking_FOUR_WHEELER,parking_NONE,parking_TWO_WHEELER,clicks
type,1.000000,0.173385,0.290261,0.306800,0.345193,0.402091,0.007067,0.695491,0.336488,0.665160,0.053992,0.246543,-0.043524,-0.277342,0.023122,0.152157,0.471538,0.070217,-0.120693,-0.449991,-0.184847
floor,0.173385,1.000000,0.739289,0.147481,0.388945,0.431016,-0.009575,0.174111,0.409899,0.233806,0.012663,0.105387,0.033449,0.012442,-0.007914,-0.037431,0.201162,0.079541,-0.029915,-0.219108,-0.039338
total_floor,0.290261,0.739289,1.000000,0.199307,0.525761,0.574919,-0.008478,0.269944,0.549500,0.345161,0.011212,0.153225,0.040248,-0.048586,-0.004511,-0.020050,0.301540,0.112735,-0.054737,-0.321961,-0.050865
furnishing,0.306800,0.147481,0.199307,1.000000,0.188268,0.270567,-0.002568,0.279377,0.171300,0.294564,0.012281,0.209588,0.037736,-0.065543,0.001701,-0.011370,0.283218,0.033509,-0.061128,-0.271795,-0.001677
gym,0.345193,0.388945,0.525761,0.188268,1.000000,0.649837,-0.008217,0.334415,0.801647,0.364114,0.003729,0.211180,0.012920,-0.074734,0.002288,0.016908,0.363300,0.114597,-0.073296,-0.377067,-0.068707
lift,0.402091,0.431016,0.574919,0.270567,0.649837,1.000000,-0.011142,0.356672,0.599912,0.391047,0.018277,0.238879,0.021880,-0.101240,-0.009056,0.019761,0.493070,0.127796,-0.099238,-0.501667,-0.071644
property_age,0.007067,-0.009575,-0.008478,-0.002568,-0.008217,-0.011142,1.000000,0.006730,-0.007703,0.019261,0.006384,-0.006799,0.001619,0.000149,0.005962,-0.002259,-0.006003,-0.003019,0.001252,0.006627,0.000672
property_size,0.695491,0.174111,0.269944,0.279377,0.334415,0.356672,0.006730,1.000000,0.332194,0.623765,0.056426,0.238314,-0.029638,-0.191724,0.025254,0.103820,0.434705,0.069823,-0.102453,-0.420356,-0.115936
swimming_pool,0.336488,0.409899,0.549500,0.171300,0.801647,0.599912,-0.007703,0.332194,1.000000,0.366477,0.004770,0.184532,0.011602,-0.069493,0.007751,0.015575,0.332783,0.113943,-0.065543,-0.349349,-0.075293
rent,0.665160,0.233806,0.345161,0.294564,0.364114,0.391047,0.019261,0.623765,0.366477,1.000000,0.088174,0.246463,-0.083994,-0.164507,0.049209,0.144525,0.428568,0.093093,-0.089106,-0.428255,-0.089538
